In [1]:
import pandas as pd
import pysolr
import requests
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('fraudulent_emails_v2.tsv', sep="\t")

In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df

,se_tags,se_tag,author_titles,attacker_offering,urgency_score,TIKA-GeoLocationParser-Content-Type,TIKA-GeoLocationParser-X-Parsed-By,TIKA-GeoLocationParser-X-TIKA:embedded_depth,TIKA-GeoLocationParser-X-TIKA:parse_time_millis,TIKA-GeoLocationParser-resourceName,...,TIKA-GeoLocationParser-dc:title,TIKA-GeoLocationParser-title,GPT2_gen_images-Phish_Iris_image-Image_filename,GPT2_gen_images-Phish_Iris_image-Image_caption-captions,GPT2_gen_images-Phish_Iris_image-Image_caption-beam_size,GPT2_gen_images-Phish_Iris_image-Image_caption-max_caption_length,GPT2_gen_images-Phish_Iris_image-Image_caption-time-read,GPT2_gen_images-Phish_Iris_image-Image_caption-time-captioning,GPT2_gen_images-Phish_Iris_image-Image_caption-time-units,GPT2_gen_images-Face_image
0,"['recon', 'social_engineering', 'phishing']",recon,['mr'],"[0.004366812227074236, 't6']",8.0,application/geotopic,"['org.apache.tika.parser.DefaultParser', 'org....",0.0,11147.0,message.geot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,['social_engineering'],social_engineering,['mr'],"[0.009481668773704172, 't6']",6.0,application/geotopic,"['org.apache.tika.parser.DefaultParser', 'org....",0.0,1579.0,message.geot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"['recon', 'social_engineering', 'phishing']",recon,['prince'],"[0.007482993197278911, 't1']",5.0,application/geotopic,"['org.apache.tika.parser.DefaultParser', 'org....",0.0,943.0,message.geot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"['recon', 'social_engineering', 'phishing']",recon,['prince'],"[0.007482993197278911, 't1']",5.0,application/geotopic,"['org.apache.tika.parser.DefaultParser', 'org....",0.0,736.0,message.geot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"['social_engineering', 'phishing']",phishing,['mrs'],"[0.004186289900575615, 't5']",11.0,application/geotopic,"['org.apache.tika.parser.DefaultParser', 'org....",0.0,1080.0,message.geot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7173,NaN,reply,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7174,NaN,reply,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7175,NaN,reply,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7176,NaN,reply,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['se_tags', 'se_tag', 'author_titles', 'attacker_offering',
       'urgency_score', 'TIKA-GeoLocationParser-Content-Type',
       'TIKA-GeoLocationParser-X-Parsed-By',
       'TIKA-GeoLocationParser-X-TIKA:embedded_depth',
       'TIKA-GeoLocationParser-X-TIKA:parse_time_millis',
       'TIKA-GeoLocationParser-resourceName', 'timestamp', 'relationship',
       'Sentiment_Analysis-neg', 'Sentiment_Analysis-neu',
       'Sentiment_Analysis-pos', 'Sentiment_Analysis-compound',
       'misspelled_ratio', 'capitalization_ratio', 'subject', 'X-TIKA:content',
       'MboxParser-from', 'MboxParser-status', 'MboxParser-x-mailer',
       'MboxParser-x-sieve', 'Content-Type',
       'GPT2_gen_images-Phish_Iris_image-Image_caption',
       'GPT2_gen_images-Attack_persona_repr', 'Falsified', 'TTR_text-ttr_text',
       'TTR_text-ttr_array', 'TIKA-GeoLocationParser-Geographic_LATITUDE',
       'TIKA-GeoLocationParser-Geographic_LONGITUDE',
       'TIKA-GeoLocationParser-Geographic_NAME',
     

In [6]:
json_docs = []
doc_id = 0
for index, row in tqdm(df.iterrows()):
    json_doc = {}
    json_doc["text"] = str(row['X-TIKA:content']).strip()  
    json_doc["id"] = str(doc_id)
    doc_id  = doc_id +1
    json_docs.append(json_doc)

In [7]:
core_name = 'fraudulent_emails'
# Create a client instance. The timeout and authentication options are not required.
solr = pysolr.Solr('http://localhost:8983/solr/'+core_name, always_commit=True, timeout=10)

In [8]:
for doc in tqdm(json_docs):
    # How you'd index data.
    solr.add([
        doc
    ])